In [1]:
import torch
from FusionNet import FusionGenerator
from model_utils_sw import MaskToBinary, DiceCoefficient

from torchvision import transforms
import torchvision.utils as v_utils
from torch.autograd import Variable
import scipy.misc

import os
import numpy as np
from PIL import Image

#=====================
# hyperparameter
INPUT_SIZE=256
#=====================

#=====================
# functions to use
# 1. image2tensor

#=====================
def image2tensor(img, cuda=False):
    img_array=np.asarray(img)
    img_array=np.expand_dims(np.expand_dims(img_array, 0), 0)
    if cuda:
        img_array=torch.Tensor(img_array).cuda()
    else:
        img_array=torch.Tensor(img_array)
    return img_array

#=====================

# load model
# model=FusionGenerator(input_nc=1, output_nc=1, ngf=32)
# model.load_state_dict(torch.load('./FusionNet.pkl'))
model=torch.load('./FusionNet.pkl').cuda()

# get data path and load data
original_dir='../train_original/'
mask_dir='../train_masks_original/'
save_dir='../train_gen_256/'
# save_dir='./save_test/'
save_label_dir='../train_masks_256/'
save_resize_origin_dir='../train_original_256/'

subjects=os.listdir(original_dir)

/home/ccids-sw/anaconda3/envs/testenv/lib/python3.5/site-packages/dicom/__init__.py:53: UserWarning: 
This code is using an older version of pydicom, which is no longer 
maintained as of Jan 2017.  You can access the new pydicom features and API 
by installing `pydicom` from PyPI.
See 'Transitioning to pydicom 1.x' section at pydicom.readthedocs.org 
for more information.

  warnings.warn(msg)


In [2]:
# pass through pre-trained model and save image to directory

for subject in subjects:
    dice_sum=0; k=0
    images=os.listdir(os.path.join(original_dir, subject))
    for i, image in enumerate(images):
        
        # get one image
        one_img=os.path.join(original_dir, subject, image)
        
        #resizing image
        one_img=Image.open(one_img)
        one_img=one_img.resize((INPUT_SIZE, INPUT_SIZE))
        
        #change to pytorch tensor
        one_img=image2tensor(one_img, cuda=True)
        
        #pass through model
        result_img=model(one_img)
        
        # if there isn't subjecy dir in save dir, make directory for subject
        if not os.path.exists(os.path.join(save_dir, subject)):
            os.mkdir(os.path.join(save_dir, subject))
            
        if not os.path.exists(os.path.join(save_label_dir, subject)):
            os.mkdir(os.path.join(save_label_dir, subject))
            
        if not os.path.exists(os.path.join(save_resize_origin_dir, subject)):
            os.mkdir(os.path.join(save_resize_origin_dir, subject))
            
        # change to numpy array again and save to suitable folder
        result_array=MaskToBinary(np.asarray(result_img[0].cpu().data))
        image_save_path=os.path.join(save_dir, subject, image)
        scipy.misc.imsave(image_save_path, result_array[0])
        
        # open mask file and calculate dice score
        # resize label image and save
        label_img=Image.open(os.path.join(mask_dir, subject, image)).resize((INPUT_SIZE, INPUT_SIZE))
        label_array=np.asarray(label_img)
        mask_save_path=os.path.join(save_label_dir, subject, image)
        scipy.misc.imsave(mask_save_path, label_array)
        
        # resize original image and save
        origin_img=Image.open(os.path.join(original_dir, subject, image)).resize((INPUT_SIZE, INPUT_SIZE))
        origin_array=np.asarray(origin_img)
        origin_save_path=os.path.join(save_resize_origin_dir, subject, image)
        scipy.misc.imsave(origin_save_path, origin_array)
        
        if result_array[0].shape != label_array.shape:
            print('Size mismatch label image size -- {}, result image size -- {} try again!'.format(label_array.shape,result_array[0].shape))

        dice_score=DiceCoefficient(np.expand_dims(label_array,0),  result_array)
#         print(image, dice_score)
        if dice_score != 0.0:
            dice_sum+=dice_score
            k += 1
    print('dice score for {} : {}'.format(subject, dice_sum / k ))
#     break

/home/ccids-sw/anaconda3/envs/testenv/lib/python3.5/site-packages/ipykernel_launcher.py:34: DeprecationWarning: `imsave` is deprecated!
`imsave` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imwrite`` instead.
/home/ccids-sw/anaconda3/envs/testenv/lib/python3.5/site-packages/ipykernel_launcher.py:41: DeprecationWarning: `imsave` is deprecated!
`imsave` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imwrite`` instead.
/home/ccids-sw/anaconda3/envs/testenv/lib/python3.5/site-packages/ipykernel_launcher.py:47: DeprecationWarning: `imsave` is deprecated!
`imsave` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imwrite`` instead.
/home/ccids-sw/Brain_segmentation/FusionNet-Pytorch/model_utils_sw.py:33: RuntimeWarning: invalid value encountered in double_scalars
  dice = np.sum(pred[gt == k] == k) * 2.0 / (np.sum(pred[pred == k] == k) + np.sum(gt[gt == k] == k))


dice score for 5233369 : nan
dice score for 5643165 : nan
dice score for 8085539 : nan
dice score for 7513852 : nan
dice score for 7654630 : nan
dice score for 5235750 : nan
dice score for 7368360 : nan
dice score for 2479047 : nan
dice score for 5408861 : nan
dice score for 5208895 : nan
dice score for 8354455 : nan
dice score for 5329449 : nan
dice score for 7354779 : nan
dice score for 5659614 : nan
dice score for 8089857 : nan
dice score for 8463790 : nan
dice score for 8262421 : nan
dice score for 8028376 : nan
dice score for 8064434 : nan
dice score for 7842502 : nan
dice score for 7354190 : nan
dice score for 4334564 : nan
dice score for 7522152 : nan
dice score for 2595099 : nan
dice score for 2659261 : nan
dice score for 8078358 : nan
dice score for 5992119 : nan
dice score for 8351755 : nan
dice score for 7746738 : nan
dice score for 8518093 : nan
dice score for 3938641 : nan
dice score for 7459615 : nan
dice score for 5319033 : nan
dice score for 7753977 : nan
dice score for

In [6]:
img=result_array

roi = np.where(img > np.median(result_array) * 1.5)
roi = list(roi)
size = img.shape
mask_binary_img = np.zeros(size)
for x, y, z in zip(roi[0], roi[1], roi[2]):
    mask_binary_img[x][y][z] = 1
mask_binary_img = mask_binary_img.astype(dtype='int')

In [7]:
(result_array - np.mean(result_array)) / np.std(result_array)

array([[[-0.18846793, -0.18846793, -0.18846793, ..., -0.18846793,
         -0.18846793, -0.18846793],
        [-0.18846793, -0.18846793, -0.18846793, ..., -0.18846793,
         -0.18846793, -0.18846793],
        [-0.18846793, -0.18846793, -0.18846793, ..., -0.18846793,
         -0.18846793, -0.18846793],
        ...,
        [-0.18846793, -0.18846793, -0.18846793, ..., -0.18846793,
         -0.18846793, -0.18846793],
        [-0.18846793, -0.18846793, -0.18846793, ..., -0.18846793,
         -0.18846793, -0.18846793],
        [-0.18846793, -0.18846793, -0.18846793, ..., -0.18846793,
         -0.18846793, -0.18846793]]])

In [41]:
mask_binary_img.shape

(1, 256, 256)

In [21]:
mask_binary_img.shape

(1, 256, 256)

In [31]:
scipy.misc.imsave('./test.jpg', mask_binary_img[0])

/home/sangwook/anaconda3/envs/torch/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: `imsave` is deprecated!
`imsave` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imwrite`` instead.
  """Entry point for launching an IPython kernel.
